## Initial parameters

In [ ]:
from blocksnet.optimization.services import (
    TPEOptimizer,
    WeightedObjective,
    WeightedConstraints,
    Facade,
    BlockSolution,
    GradientChooser,
    AreaSolution,
)
from blocksnet.enums import LandUse
import numpy as np
import pandas as pd
import geopandas as gpd
import ast
import csv

: 

#### Services

In [2]:
BASIC_SERVICES = {
    "kindergarten": 0.2,
    "school": 0.2,
    "pharmacy": 0.2,
    "polyclinics": 0.3,
    "convenience": 0.2,   
    "cafe": 0.15,
    "playground": 0.2,
    "post": 0.1,
    "hairdresser": 0.1
}
ADVANCED_SERVICES = {
    "hospital": 0.5,
    "supermarket": 0.4,
    "restaurant": 0.3,
    "park": 0.8,
    "bank": 0.3,
    "fuel": 0.25,
    "police": 0.2,
    "cinema": 0.2,
    "market": 0.2,
    "religion": 0.5,
    "multifunctional_center": 0.1
}
COMFORT_SERVICES = {
    "mall": 0.3,   
    "swimming_pool": 0.4,
    "theatre": 0.2,
    "museum": 0.2,
    "university": 0.2,
    "sanatorium": 0.2,
}

### Block id for profiling

In [3]:
PROFILING_ID = 1536 

### City Initialization

In [4]:
blocks = gpd.read_parquet('./../../data/blocks.parquet')
columns = [c for c in blocks.columns if 'capacity_' in c]
blocks = blocks[['geometry', 'population', 'site_area', *columns]].copy()
blocks.head(3)

,geometry,population,site_area,capacity_convenience,capacity_cafe,capacity_mall,capacity_pharmacy,capacity_bank,capacity_fuel,capacity_pitch,...,capacity_bus_station,capacity_bus_stop,capacity_pier,capacity_animal_shelter,capacity_prison,capacity_landfill,capacity_plant_nursery,capacity_greenhouse_complex,capacity_warehouse,capacity_machine_building_plant
id,,,,,,,,,,,,,,,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",565.0,804466.712114,208.0,0.0,2216.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",42.0,23173.129862,811.0,25.0,0.0,0.0,148.0,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",19.0,363005.815802,0.0,0.0,0.0,0.0,0.0,501.0,0.0,...,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
accessibility_matrix = pd.read_pickle('./../../data/accessibility_matrix.pickle')
accessibility_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,16310,16311,16312,16313,16314,16315,16316,16317,16318,16319
0,0.000000,18.718750,9.234375,17.21875,30.296875,27.390625,30.671875,23.281250,78.6875,81.2500,...,76.5625,102.5625,97.5000,132.250,130.8750,142.125,247.625,228.625,247.625,124.6250
1,17.859375,0.000000,19.015625,7.65625,20.281250,25.250000,25.234375,15.765625,76.8750,79.4375,...,74.6875,100.7500,95.6875,130.500,129.0000,140.250,245.875,226.875,245.875,112.5000
2,9.234375,20.796875,0.000000,20.21875,27.968750,25.062500,28.343750,20.953125,71.8750,74.4375,...,69.7500,95.8125,90.6875,125.500,124.0625,135.375,240.875,221.875,240.875,124.6250
3,15.937500,7.449219,18.421875,0.00000,21.109375,24.984375,27.781250,16.593750,76.3125,78.8125,...,74.1250,100.1875,95.0625,129.875,128.5000,139.750,245.250,226.250,245.250,113.3750
4,31.906250,19.765625,31.734375,21.28125,0.000000,21.781250,23.125000,15.429688,89.5625,92.1250,...,87.4375,113.5000,108.3750,143.125,141.7500,153.000,258.500,239.500,258.500,108.3125


### Service weights

In [6]:
def get_service_weight(service_name):
    if service_name in BASIC_SERVICES:
        return 0.5714 * BASIC_SERVICES[service_name] / sum(BASIC_SERVICES.values())
    elif service_name in ADVANCED_SERVICES:
        return 0.2857 * ADVANCED_SERVICES[service_name] / sum(ADVANCED_SERVICES.values())
    elif service_name in COMFORT_SERVICES:
        return 0.1429 * COMFORT_SERVICES[service_name] / sum(COMFORT_SERVICES.values())
    return 0

chosen_service_types = set(BASIC_SERVICES) | set(ADVANCED_SERVICES) | set(COMFORT_SERVICES)
service_weights = {service_type: get_service_weight(service_type) for service_type in chosen_service_types if f'capacity_{service_type}' in blocks.columns}

### Save service to csv

In [7]:
def save_max_capacities(land_use: LandUse, facade: Facade):
    with open(f'{land_use.name}_max_capacities.csv', 'w', newline='') as csvfile:
        capacities = facade.get_max_capacities(PROFILING_ID)

        writer = csv.writer(csvfile)
        writer.writerow(["Service type", "Capacity required"])  # Write header row
        
        for service, capacity in capacities.items():
            writer.writerow([service, capacity])

In [8]:
import os
import shutil
from pathlib import Path

def move_csv_to_results():
    results_dir = Path("./results")
    results_dir.mkdir(exist_ok=True)
    
    csv_files = [f for f in os.listdir() if f.endswith('.csv')]
    
    for csv_file in csv_files:
        src = Path(csv_file)
        dest = results_dir / csv_file
        shutil.move(str(src), str(dest))

## Land use provision with selected services

In [ ]:
from tqdm import tqdm

provision_data = {land_use: dict() for land_use in LandUse}

for land_use in LandUse:
    blocks_lu = dict()

    blocks_lu[PROFILING_ID] = land_use

    var_adapter = AreaSolution(blocks_lu)

    facade = Facade(
        blocks_lu=blocks_lu,
        blocks_df=blocks,
        accessibility_matrix=accessibility_matrix,
        var_adapter=var_adapter,
    )

    for service_type, weight in tqdm(service_weights.items()):
        facade.add_service_type(service_type, weight, blocks.rename(columns={f'capacity_{service_type}': 'capacity'})[['capacity']])

    start_prov = facade.start_provisions

    objective = WeightedObjective(num_params=facade.num_params, facade=facade, weights=service_weights, max_evals=50)

    constraints = WeightedConstraints(num_params=facade.num_params, facade=facade)

    tpe_optimizer = TPEOptimizer(
        objective=objective, constraints=constraints, vars_chooser=SimpleChooser(facade, facade.num_params, 5)
    )

    best_x, best_val, perc, func_evals = tpe_optimizer.run(max_runs=1000, timeout=60000, initial_runs_num=1)

    solution_df = facade.solution_to_services_df(best_x)
    solution_df.to_csv(f'{land_use.name}_services.csv')

    save_max_capacities(land_use, facade)

    df = pd.read_csv('tpe_provisions.csv')
    best_provisions = None
    max_val = 0
    for index, row in df.iterrows():
        if max_val < row['best_val']:
            max_val = row['best_val']
            best_provisions = row['provisions']
    best_provisions = ast.literal_eval(best_provisions)
    for service in facade._chosen_service_types:
        if service in best_provisions.keys():
            opt = best_provisions[service]
        else:
            opt = start_prov[service]
        start = start_prov[service]
        provision_data[land_use][service] = (start, opt)

    move_csv_to_results()
    break

100%|██████████| 25/25 [00:43<00:00,  1.73s/it]
[I 2025-04-20 01:41:36,495] A new study created in memory with name: no-name-3daf991b-e4d8-47df-99cb-d01807d6ce6b


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2025-04-20 01:41:36,500] Trial 0 finished with value: 0.034283999999999995 and parameters: {}. Best is trial 0 with value: 0.034283999999999995.


### Save provisions data

In [10]:
with open('./results/provisions_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Land use", "Service type", "Start provisions", "Opt. provisions", "Delta provisions"])  # Write header row
    
    for land_use, services in provision_data.items():
        for service_type, (start_prov, opt_prov) in services.items():
            writer.writerow([land_use, service_type, start_prov, opt_prov, opt_prov - start_prov])